In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import utils
import models
import data_mappers

In [ ]:
import importlib
importlib.reload(utils)
importlib.reload(models)
importlib.reload(data_mappers)

In [ ]:
ours = sorted(utils.enumerate_recording_paths('../dataset/raw/*record*'))
ours += sorted(utils.enumerate_recording_paths('../dataset/raw/benni*/*record*'))
old = sorted(utils.enumerate_recording_paths('../dataset/raw/old/*record*'))
recs = sorted(utils.enumerate_recording_paths('/Users/pabloprietz/Dropbox/DCAITI_sharedfolder/Recordings/Kevin/rec*'))

In [ ]:
old

In [ ]:
def plot_rec(r):
    f = pd.read_msgpack(os.path.join(r, 'merged.msg'))
    fig, ax = plt.subplots(figsize=(16, 4))
    for i, dim in enumerate((#'Accelerometer_val_x',
                             #'Accelerometer_val_y',
                             #'Accelerometer_val_z',
                             'Normalized_Accelerometer_val_x',
                             'Normalized_Accelerometer_val_y',
                             'Normalized_Accelerometer_val_z')):
#         plt.plot(f['event_ts'], f[dim], 'x', label=dim)
        plt.plot(f[dim], 'x', label=dim)

#     print((f['event_ts'].iloc[-1] - f['event_ts'].iloc[0]) / f['event_ts'].shape[0])

    plt.plot(f['label'], '|', c='r', label='label')
#     plt.plot(f['event_ts'], f['label'], '|', c='r', label='label')
    plt.title(r)
    plt.legend()
    
def calc_fps(r):
    f = pd.read_msgpack(os.path.join(r, 'merged.msg'))
    return f['event_ts'].shape[0] / (f['event_ts'].iloc[-1] - f['event_ts'].iloc[0])

def calc_dur(r):
    f = pd.read_msgpack(os.path.join(r, 'merged.msg'))
    return f['event_ts'].iloc[-1] - f['event_ts'].iloc[0]

In [ ]:
dur_ours = 0
for r in ours:
    dur_ours += calc_dur(r)
print(f'Recording length: {dur_ours/60:.2f}mins')

In [ ]:
for r in ours:
    plot_rec(r)

In [ ]:
wc = data_mappers.Window_Creator(recs)

In [ ]:
plt.hist(wc.labels[wc.training_idc], bins=2)
plt.hist(wc.labels[wc.testing_idc], bins=2)
plt.ylabel('Number of Windows')
plt.xticks([.25, .75], ['not walking', 'walking'])

In [ ]:
R = models.Recording(ours[0])

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 14})

FIGSIZE = (16, 2)

R.merge()
R.rotate_accelerometer()
R.normalize_accelerometer(8.)

In [ ]:
def make_plt(dims, title, xlabel, ylabel):
    fig, ax = plt.subplots(figsize=FIGSIZE)
    for i, dim in enumerate(dims):
        plt.plot(R.merged['event_ts'] - R.merged['event_ts'][0], R.merged[dim], 'x', label=dim[-1])

    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()

make_plt(('Accelerometer_val_x', 'Accelerometer_val_y', 'Accelerometer_val_z'),
         'Raw accelerometer', 'Arrival time [sec]', 'Acceleration [$m/s^2$]')

make_plt(('Rotated_Accelerometer_val_x', 'Rotated_Accelerometer_val_y', 'Rotated_Accelerometer_val_z'),
         'Rotated accelerometer', 'Arrival time [sec]', 'Acceleration [$m/s^2$]')
    
fig, ax = plt.subplots(figsize=FIGSIZE)
for i, dim in enumerate(('Rotated_Accelerometer_val_x',
                         'Rotated_Accelerometer_val_y')):
    plt.plot(R.merged['event_ts'] - R.merged['event_ts'][0], R.merged[dim], 'x', label=dim[-1])
plt.plot(R.merged['event_ts'] - R.merged['event_ts'][0], R.merged['Rotated_Accelerometer_val_z'] - 9.8, 'x', label=dim[-1])
plt.title('Rotated accelerometer without gravitation')
plt.xlabel('Arrival time [sec]')
plt.ylabel('Acceleration [$m/s^2$]')
plt.legend()
    
make_plt(('Normalized_Accelerometer_val_x', 'Normalized_Accelerometer_val_y', 'Normalized_Accelerometer_val_z'),
         'Normalized accelerometer', 'Arrival time [sec]', 'Normalized acc. [$m/8s^2$]')

In [ ]:
from matplotlib import collections  as mc

def plot_feet(r):
    fig, ax = plt.subplots(figsize=FIGSIZE)
    for i, dim in enumerate(('val_x', 'val_y', 'val_z')):
        plt.plot(r.Accelerometer['arrival_ts'],
                 r.Accelerometer[dim], 'x', c=(1-(i + 2)/6, 1-(i + 2)/6, 1-(i + 2)/6, .2))
    
    zipped_times = zip(r.RightFootSensor['arrival_ts'], r.RightFootSensor['duration'] * 1e-3)
    right_lines = [[(arr - dur, 0), (arr, 0)] for arr, dur in zipped_times]

    zipped_times = zip(r.LeftFootSensor['arrival_ts'], r.LeftFootSensor['duration'] * 1e-3)
    left_lines = [[(arr - dur, 0), (arr, 0)] for arr, dur in zipped_times]

    lc = mc.LineCollection(right_lines, linewidths=5, colors='red', zorder=10, label='Right')
    ax.add_collection(lc)
    lc = mc.LineCollection(left_lines, linewidths=5, colors='blue', zorder=10, label='Left')
    ax.add_collection(lc)
    ax.autoscale()
    plt.title('Raw foot sensors')
    plt.xlabel('Arrival timestamps')
    plt.ylabel('Acceleration [$m/s^2$]')
    plt.legend()
    
plot_feet(R)

plt.figure(figsize=FIGSIZE)
dims = ('Normalized_Accelerometer_val_x', 'Normalized_Accelerometer_val_y', 'Normalized_Accelerometer_val_z')
title, xlabel, ylabel = 'Merged data', 'Arrival time [sec]', 'Normalized acc. [$m/8s^2$]'
for i, dim in enumerate(dims):
    plt.plot(R.merged['event_ts'] - R.merged['event_ts'][0], R.merged[dim], 'x', label='')

plt.plot(R.merged['event_ts'] - R.merged['event_ts'][0], R.merged['label'], '|', c='r', label='Labels')
    
plt.title(title)
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.legend()

In [ ]:
print('fps,duration[sec]')
for r in ours:
    print(f'{calc_fps(r)},{calc_dur(r)}')